In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [3]:
# The code was removed by DSX for sharing.

/usr/local/src/conda3_runtime.v25/4.1.1/lib/python3.5/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/src/conda3_runtime.v25/4.1.1/lib/python3.5/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/src/conda3_runtime.v25/4.1.1/lib/python3.5/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/src/conda3_runtime.v25/4.1.1/lib/python3.5/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_defau

,Text
0,"The ""Big Brother"" of Singapore football will b..."
1,Mahfizur Rahman watched his friends turn to cr...
2,"The going has been tough, but the Football Ass..."
3,Having pushed reigning world and European cham...
4,SINGAPORE - Registration for the Standard Char...


In [4]:
docs_train=[]
for idx, row in df_data_1.iterrows():
    #print (row['Label'])
    
    #tokens = nltk.word_tokenize(row['Data'])
   
    docs_train.append(row['Text'])
    #labels_train.append(row['Label'])
    #text2_train = nltk.Text(tokens)
print (docs_train)

['The "Big Brother" of Singapore football will be back, but not immediately, and not for long. In an exclusive interview with The New Paper, Persib Bandung striker Noh Alam Shah said he has agreed to sign a short-term deal with former club Tampines Rovers until the end of the season.\nBut the 31-year-old said: "Beyond that, I feel my future is still in Indonesia.\n"I feel really appreciated here. Four Indo clubs already made me offers for the next season, which starts next January."\nThe move to Singapore still hinges on whether Tampines can secure his medical documents and International Transfer Certificate from the Indonesia FA before the transfer window closes today, although the Stags are optimistic.\nIf there are no surprises, Alam Shah will return to Singapore after July 11, after Persib play their final Indonesia Super League (ISL) match against champions Sriwijaya.\nSaid the striker: "Tampines have always been very close to my heart and I\'m thankful to \'Boss\' (Tampines chair

In [5]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [6]:
stopwords = nltk.corpus.stopwords.words('english')
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [7]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in docs_train:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [8]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 2098 items in vocab_frame


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(docs_train) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 159 ms, sys: 549 µs, total: 160 ms
Wall time: 186 ms
(5, 200)


In [10]:
terms = tfidf_vectorizer.get_feature_names()

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 2

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()